In [69]:
%pip install -r requirements.txt

^C
Note: you may need to restart the kernel to use updated packages.


     ------------------------------------- 491.5/491.5 kB 10.2 MB/s eta 0:00:00
  Using cached pyffx-0.3.0.tar.gz (3.0 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached transformers-4.51.3-py3-none-any.whl (10.4 MB)
     -------------------------------------- 212.5/212.5 MB 6.0 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 8.4 MB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 7.2 MB/s eta 0:00:00
  Using cached accelerate-1.6.0-py3-none-any.whl (354 kB)
  Using cached tenseal-0.3.16-cp311-cp311-win_amd64.whl (2.2 MB)
  Using cached opacus-1.5.3-py3-none-any.whl (251 kB)
  Using cached spacy-3.8.5-cp311-cp311-win_amd64.whl (12.2 MB)
  Using cached stanza-1.10.1-py3-none-any.whl (1.1 MB)
  Using cached flair-0.15.1-py3-none-any.whl (1.2 MB)
  Using cached sentence_transformers-4.1.0-py3-none-any.whl (345 kB)
     -------------------------------------- 158.3/158.3 kB 

  DEPRECATION: sqlitedict is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  DEPRECATION: pptree is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  DEPRECATION: docopt is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found a

In [78]:
import pandas as pd

df = pd.read_csv("output/sanitized_test_output.csv")
df

,text,label,sanitized_text,sanitized_pct,perturbed_label
0,John Doe's email is john.doe@example.com and h...,1,Patrick Thompson email is UqACt^NA!#en[2Qkuyy....,96.97,3
1,Alice paid with credit card 4111-1111-1111-111...,0,Denise Haas paid with credit card <REMOVED> on...,100.00,0
2,Michael called 123-456-7890 to book a flight t...,2,Casey Stark called 123-456-7890 to book a flig...,96.36,1
3,Contact jane.smith87@gmail.com or +1-234-567-8...,3,Contact &)xFuQy01ZvRNx>0RJQDDL or +1-234-567-8...,30.99,1
4,Ben paid 23 dollar using his credit card 4111-...,2,Anna Lopez paid 1000 dollar using his credit c...,99.07,1


In [66]:
df['text'][4]

'Ben paid 23 dollar using his credit card 4111-1111-1111-1111 on 5pm buying 3 packets of apples in New York.'

In [67]:
df['sanitized_text'][4]

'Dwayne Reynolds paid 1000 dollar using his credit card <REMOVED> on <REMOVED> buying 3 packets of apples in [GPE_001].'

In [73]:
import math
import re
import nltk
from nltk.corpus import words

nltk.download('words')
english_words = set(words.words())

def shannon_entropy(token):
    from collections import Counter
    prob = [freq / len(token) for freq in Counter(token).values()]
    return -sum(p * math.log2(p) for p in prob)

def is_suspicious(token):
    entropy = shannon_entropy(token)
    is_dict_word = token.lower() in english_words
    contains_weird_chars = bool(re.search(r'[^a-zA-Z0-9]', token))
    digit_ratio = sum(c.isdigit() for c in token) / (len(token) + 1e-5)
    
    # thresholds may require tuning
    if entropy > 3.5 and (not is_dict_word or digit_ratio > 0.3 or contains_weird_chars):
        return True
    return False


[nltk_data] Downloading package words to
[nltk_data]     C:\Users\cihui\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\words.zip.


In [74]:
sentence = "In a panel discussion, Todd Lang, Chief Privacy Officer at Huawei, warned of the potential misuse of location data, citing the February 2,1965 data breach at Huawei, where over 500,000 user profiles—including email addresses like &)xFuQy01ZvRNx>0RJQDDL and partial social security numbers—were leaked online."
tokens = sentence.split()
heuristic_flags = [is_suspicious(tok) for tok in tokens]

In [77]:
for i in range(len(tokens)):
    if heuristic_flags[i] == True:
        
        tokens[i] = "HERE"
        
' '.join(tokens)

'In a panel discussion, Todd Lang, Chief Privacy Officer at Huawei, warned of the potential misuse of location data, citing the February 2,1965 data breach at Huawei, where over 500,000 user HERE email addresses like HERE and partial social security numbers—were leaked online.'

In [ ]:
%pip install transformers termcolor torch torchvision torchaudio

In [ ]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForTokenClassification
from termcolor import colored
import re
from nltk.corpus import words
english_vocab = set(words.words())

MODEL_NAME = "perfectsquare123/encrypted-token-detector"
CONFIDENCE_THRESHOLD = 0.85

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForTokenClassification.from_pretrained(MODEL_NAME)
id2label = model.config.id2label

def is_false_positive_candidate(token):
    if len(token) <= 1:
        return True
    if re.match(r"\[?[A-Z]{2,}[0-9]{3,}\]?", token): # for tokenized [PERSON001]
        return True
    if token.lower() in english_vocab:
        return True
    if re.match(r"^[A-Z][a-z]+$", token):  # common names
        return True
    if re.match(r"<REDACT>", token): # for redaction
        return True
    if re.match(r".+'s$|.+'s$|.+s'$", token): # e.g., "John's", "boss'"
        return True
    if re.match(r"^(?:\+65[- ]?)?[89]\d{7}$", token) or re.match(r"^(\+1[- ]?)?(\(?\d{3}\)?[- ]?)?\d{3}[- ]?\d{4}$", token): # sg or US phone number
        return True
    if re.match(r"^(\d{4}[- ]?){3}\d{4}$", token): # credit card number
        return True
    return False

def highlight_encrypted_tokens_with_confidence(sentence):
    tokens = sentence.strip().split()
    inputs = tokenizer(tokens, return_tensors="pt", is_split_into_words=True, truncation=True)
    outputs = model(**inputs)
    logits = outputs.logits
    probs = F.softmax(logits, dim=-1)
    predictions = torch.argmax(probs, dim=-1).squeeze().tolist()

    word_ids = inputs.word_ids()
    prev_word_idx = None

    print("\n🔍 Detected Tokens:")
    for i, token_id in enumerate(inputs["input_ids"][0]):
        word_idx = word_ids[i]
        if word_idx is None or word_idx == prev_word_idx:
            continue

        token = tokens[word_idx]
        label_id = predictions[i]
        label = id2label[label_id]
        confidence = probs[0, i, label_id].item()

        if label == "B-ENC" and confidence >= CONFIDENCE_THRESHOLD and not is_false_positive_candidate(token):
            print(colored(f"{token}({confidence:.2f})", "red", attrs=["bold"]), end=" ")
        else:
            print(token, end=" ")

        prev_word_idx = word_idx
    print()



c:\Users\cihui\Documents\GitHub\speakWithoutLeaks-data-level-privacy-preserving-pipeline-for-llm\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Interactive loop
if __name__ == "__main__":
    print("Enter a sentence to detect encrypted tokens with confidence (Ctrl+C to exit):")
    while True:
        try:
            sentence = input("> ")
            highlight_encrypted_tokens_with_confidence(sentence)
        except KeyboardInterrupt:
            print("\nExiting.")
            break

Enter a sentence to detect encrypted tokens with confidence (Ctrl+C to exit):

🔍 Detected Tokens:
John Doe's pencil case sent to [PERSON001] with sbdfib()(&&^%Sjbs99180(1.00) with +6589200549 phone and i think DKJBS8898*(&( 0089JS(1.00) that <REDACT> 
